In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# PLot the dataset

## Data overview

In [ ]:
df = pd.read_csv('../input/insurance/insurance.csv')
df.head(10),df.shape

In [ ]:
# Check some general information:
df.info() 

In [ ]:
# Decribe the data set
df.describe()

## Data visuallization

In [ ]:
List_input = ['age','bmi','children','charges']

In [ ]:
bmi = df['bmi']
age = df['age']
charges  = df['charges']
children = df['children']

In [ ]:
fig, ((ax1,ax2),(ax3,ax4))  = plt.subplots(nrows= 2, ncols=2, figsize =(15,10))

#Plot data to each axis:
ax1.scatter(age,charges, color = 'b');
ax2.scatter(bmi,charges, color = 'g');
ax3.scatter(children,charges, color = 'pink');
ax4.scatter(age,bmi, color = 'red');
            
ax1.set(title = 'Relationship between age vs charge', xlabel= 'age', ylabel='charges');
ax2.set(title = 'Relationship between bmi vs charge', xlabel= 'bmi', ylabel='charges');
ax3.set(title = 'Relationship between children vs charge', xlabel= 'children', ylabel='charges');
ax4.set(title = 'Relationship between bmi vs age', xlabel= 'age', ylabel='bmi');


##### there are three group in the age

In [ ]:
age_group1 = df.loc[(df['charges'] > 0) & (df['charges'] < 13000)]
age_group2 = df.loc[(df['charges'] > 13000) & (df['charges'] < 28000)]
age_group3 = df.loc[(df['charges'] > 28000)]

In [ ]:
def Exploxring_the_data(dataframe): 
    #Prepare the data
    bmi = dataframe['bmi']
    age = dataframe['age']
    charges  = dataframe['charges']
    children = dataframe['children']
    region = dataframe['region']
    smoking = dataframe['smoker']
    #Some looks
    a = dataframe.info()
    list_infor = [bmi.value_counts(),age.value_counts(),charges.value_counts(),children.value_counts(),region.value_counts(),smoking.value_counts()]

    fig, (ax1,ax2,ax3)  = plt.subplots(nrows= 1, ncols=3, figsize =(15,5))

    #Plot data to each axis:
    ax1.scatter(bmi,charges, color = 'g');
    ax2.scatter(children,charges, color = 'pink');
    ax3.scatter(age,bmi, color = 'red');

    ax1.set(title = 'Relationship between bmi vs charge', xlabel= 'bmi', ylabel='charges');
    ax2.set(title = 'Relationship between children vs charge', xlabel= 'children', ylabel='charges');
    ax3.set(title = 'Relationship between bmi vs age', xlabel= 'age', ylabel='bmi');
    
    return a,list_infor

In [ ]:
Exploxring_the_data(age_group1)

In [ ]:
Exploxring_the_data(age_group2)

In [ ]:
Exploxring_the_data(age_group3)

In [ ]:
pattern_group2_no_smoke = age_group2.loc[(age_group2['smoker']=='no')]
pattern_group2_smoke = age_group2.loc[(age_group2['smoker']=='yes')]

bmi_no_smoke = pattern_group2_no_smoke['bmi']
bmi_smoke = pattern_group2_smoke['bmi']
charges_no_smoke = pattern_group2_no_smoke['charges']
charges_smoke = pattern_group2_smoke['charges']


pattern_group2_no_smoke['bmi'].value_counts(),pattern_group2_smoke['bmi'].value_counts()

In [ ]:
fig, (ax1,ax2)  = plt.subplots(nrows= 1, ncols=2, figsize =(15,5))

#Plot data to each axis:
ax1.scatter(bmi_no_smoke,charges_no_smoke, color = 'g');
ax2.scatter(bmi_smoke,charges_smoke, color = 'pink');


ax1.set(title = 'Relationship between bmi vs charge of no smoke', xlabel= 'bmi', ylabel='charges');
ax2.set(title = 'Relationship between bmi vs charge of smoke', xlabel= 'bmi', ylabel='charges');
    

### We can slightly conclude that: the people who have higher  medical charges tend to be the ones who smoke and overweight ( the bmi > 25)

### Preprocess the data

In [ ]:
X = df.drop (['charges'],axis = 1)
y = df['charges']

X

In [ ]:
list_name = ['sex','smoker','region']
X = pd.get_dummies(X, prefix = list_name)
X

In [ ]:
X_new = X.drop(['sex_male','smoker_yes','region_southwest'],axis = 1)
X_new,y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape,X_val.shape

In [ ]:
X_train.iloc[:,:3]

## Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train.iloc[:,0:3] = sc.fit_transform(X_train.iloc[:,0:3])
X_test.iloc[:,0:3] = sc.transform(X_test.iloc[:,0:3])
X_val.iloc[:,0:3] = sc.transform(X_val.iloc[:,0:3])

In [ ]:
y_train = tf.expand_dims(y_train,1, name=None)
y_test = tf.expand_dims(y_test,1, name=None)
y_val = tf.expand_dims(y_val,1, name=None)

y_train.shape

In [ ]:
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train)
y_test = sc_y.transform(y_test)
y_val = sc_y.transform(y_val)

In [ ]:
X_train.shape

## Min_Max_scaler

from sklearn.preprocessing import MinMaxScaler
mn = MinMaxScaler()
X_train.iloc[:,0:3] = mn.fit_transform(X_train.iloc[:,0:3])
X_test.iloc[:,0:3] = mn.transform(X_test.iloc[:,0:3])
X_val.iloc[:,0:3] = mn.transform(X_val.iloc[:,0:3])

y_train = tf.expand_dims(y_train,1, name=None)
y_test = tf.expand_dims(y_test,1, name=None)
y_val = tf.expand_dims(y_val,1, name=None)

y_train.shape

mn_y = MinMaxScaler()
y_train = mn_y.fit_transform(y_train)
y_test = mn_y.transform(y_test)
y_val = mn_y.transform(y_val)

### Build the neural network

In [ ]:
#1. Create the model:
model = tf.keras.Sequential([
      tf.keras.layers.Dense(856, activation='relu'),
      #tf.keras.layers.Dense(700, activation='relu'),
      tf.keras.layers.Dense(400, activation='relu'),
      #tf.keras.layers.Dense(300, activation='relu'),
      #tf.keras.layers.Dense(200,activation='relu'),
      tf.keras.layers.Dropout(0.3),
      #tf.keras.layers.Dense(150,activation='relu'),
      #tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(100,activation='relu'),
      #tf.keras.layers.Dense(10,activation='relu'),
      tf.keras.layers.Dense(1)
], name='Excersice_1')

#2. Compile the model 
model.compile(loss=tf.keras.losses.mae, 
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics =['mse'])

# 3. Fit the model
model.fit(X_train,y_train,validation_data = (X_val, y_val),epochs = 200, batch_size = 60)

In [ ]:
model.summary()

# Evaluate the model

In [ ]:
y_pred = model.predict(X_test)
y_pred.shape

In [ ]:
y_pred_1 = tf.squeeze(y_pred)
y_test = tf.squeeze(y_test)
y_pred_1.shape,y_test.shape

In [ ]:
mae = tf.metrics.mean_absolute_error(y_true=y_test, 
                                     y_pred=y_pred_1)
a = mae
a

In [ ]:
# Calculate the MSE
mse = tf.metrics.mean_squared_error(y_true=y_test,
                                    y_pred=y_pred_1)
mse

In [ ]:
# R_score
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

## Save the model

In [ ]:
#model.save('Model_ex1_1')

# Try SVR model

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)
y_pred.shape

In [ ]:
# R_score
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)